In [1]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.callbacks import LearningRateScheduler
import math

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
Z_eff = pd.read_csv('data_Z_eff.csv',header=None)
R_eff = pd.read_csv('data_R_eff.csv',header = None)
R_org_eff = pd.read_csv('data_R_org_eff.csv',header = None)
group_ind = pd.read_csv('data_group_ind.csv',header =None)

In [4]:
t_train_start = list(range(0,41*12+1,12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x+12 for x in t_test_start]
t_test_end[41]=678

In [5]:
train_start = group_ind.iloc[t_train_start][0].to_list()
train_end = group_ind.iloc[t_train_end][0].to_list()
test_start = group_ind.iloc[t_test_start][0].to_list()
test_end = group_ind.iloc[t_test_end][0].to_list()
val_start = group_ind.iloc[t_val_start][0].to_list()
val_end = group_ind.iloc[t_val_end][0].to_list()                         

In [6]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [7]:
y_pred = []
y_pred_val = []
y_pred_test = []
loss = []
model_list = []
y_org = []
y_org_val = []
y_org_test = []
y_reff = []
y_reff_val = []
y_reff_test = []
for retrain_idx in range(42):
    retrain_idx
    X = Z_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Z_eff.loc[val_start[retrain_idx]:val_end[retrain_idx],X_idx]
    X_test = Z_eff.loc[test_start[retrain_idx]:test_end[retrain_idx],X_idx]
    y =R_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val = R_eff.loc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test = R_eff.loc[test_start[retrain_idx]:test_end[retrain_idx],:]
    yorg = R_org_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val = R_org_eff.loc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test=R_org_eff.loc[test_start[retrain_idx]:test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(8,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mse')
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=64,epochs=600,verbose=1,callbacks=[early_stop])
    y_pred.append(model.predict(X))
    y_pred_val.append(model.predict(X_val))
    y_pred_test.append(model.predict(X_test))
    model_list.append(model)
    loss.append(min(history.history['val_loss']))
    model_list.append(model)
    yorg_test['prediction'] = model.predict(X_test)
    y_org_test.append(yorg_test)
    
    


    
    
    

Epoch 1/600
3212/3212 [==============================] - 9s 2ms/step - loss: 0.0181 - val_loss: 0.0084
Epoch 2/600
3212/3212 [==============================] - 7s 2ms/step - loss: 0.0074 - val_loss: 0.0083
Epoch 3/600
3212/3212 [==============================] - 8s 2ms/step - loss: 0.0073 - val_loss: 0.0082
Epoch 4/600
3212/3212 [==============================] - 7s 2ms/step - loss: 0.0073 - val_loss: 0.0084
Epoch 5/600
3212/3212 [==============================] - 7s 2ms/step - loss: 0.0072 - val_loss: 0.0084
Epoch 00005: early stopping


<ipython-input-7-3da047f01ae9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorg_test['prediction'] = model.predict(X_test)


Epoch 1/600
3504/3504 [==============================] - 10s 2ms/step - loss: 0.0237 - val_loss: 0.0078
Epoch 2/600
3504/3504 [==============================] - 8s 2ms/step - loss: 0.0079 - val_loss: 0.0076
Epoch 3/600
3504/3504 [==============================] - 8s 2ms/step - loss: 0.0078 - val_loss: 0.0076
Epoch 4/600
3504/3504 [==============================] - 8s 2ms/step - loss: 0.0078 - val_loss: 0.0075
Epoch 5/600
3504/3504 [==============================] - 8s 2ms/step - loss: 0.0077 - val_loss: 0.0075
Epoch 6/600
3504/3504 [==============================] - 8s 2ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 00006: early stopping


<ipython-input-7-3da047f01ae9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorg_test['prediction'] = model.predict(X_test)


Epoch 1/600
3704/3704 [==============================] - 10s 2ms/step - loss: 0.0160 - val_loss: 0.0072
Epoch 2/600
3704/3704 [==============================] - 9s 2ms/step - loss: 0.0084 - val_loss: 0.0074
Epoch 3/600
3704/3704 [==============================] - 8s 2ms/step - loss: 0.0084 - val_loss: 0.0071
Epoch 4/600
3704/3704 [==============================] - 9s 2ms/step - loss: 0.0083 - val_loss: 0.0072
Epoch 5/600
3704/3704 [==============================] - 9s 2ms/step - loss: 0.0082 - val_loss: 0.0071
Epoch 6/600
3704/3704 [==============================] - 9s 2ms/step - loss: 0.0082 - val_loss: 0.0072
Epoch 7/600
3704/3704 [==============================] - 8s 2ms/step - loss: 0.0082 - val_loss: 0.0072
Epoch 00007: early stopping


...

<ipython-input-7-3da047f01ae9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorg_test['prediction'] = model.predict(X_test)


Epoch 1/600
6199/6199 [==============================] - 14s 2ms/step - loss: 0.0099 - val_loss: 0.0073
Epoch 2/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0080 - val_loss: 0.0073
Epoch 3/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0080 - val_loss: 0.0073
Epoch 4/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0080 - val_loss: 0.0073
Epoch 5/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0073
Epoch 6/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0073
Epoch 7/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0073
Epoch 8/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0073
Epoch 9/600
6199/6199 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0073
Epoch 10/600
6199/6199 [==============================] - 13s 2m

<ipython-input-7-3da047f01ae9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorg_test['prediction'] = model.predict(X_test)


Epoch 1/600
6049/6049 [==============================] - 14s 2ms/step - loss: 0.0095 - val_loss: 0.0076
Epoch 2/600
6049/6049 [==============================] - 13s 2ms/step - loss: 0.0079 - val_loss: 0.0078
Epoch 3/600
6049/6049 [==============================] - 13s 2ms/step - loss: 0.0080 - val_loss: 0.0077
Epoch 00003: early stopping


<ipython-input-7-3da047f01ae9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yorg_test['prediction'] = model.predict(X_test)


In [8]:
portf_ret_nn_test = []
for retrain_idx in range(42):
    for i in range(t_test_start[retrain_idx],t_test_end[retrain_idx]):
        tesdd = y_org_test[retrain_idx].loc[group_ind[0][i]:group_ind[0][i+1],]
        list_buy = [idx for idx, val in enumerate(tesdd['prediction']) if val > np.percentile(tesdd['prediction'],90)]
        list_sell = [idx for idx, val in enumerate(tesdd['prediction']) if val < np.percentile(tesdd['prediction'],10)]
        portf_ret_nn_test.append(np.mean([tesdd[0].to_list()[i] for i in list_buy]) - np.mean([tesdd[0].to_list()[i] for i in list_sell]))

/apps/spack/scholar/fall20/apps/anaconda/2020.11-py38-gcc-4.8.5-djkvkvk/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/apps/spack/scholar/fall20/apps/anaconda/2020.11-py38-gcc-4.8.5-djkvkvk/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
portfolio = pd.DataFrame(portf_ret_nn_test,columns=['monthly_return'])

In [11]:
portfolio['monthly_return'].mean()/portfolio['monthly_return'].std()

1.5841183058099282

In [ ]:
portfolio.to_csv('portfolio.csv',index=False)